In [5]:
import numpy as np
def wavemaker(h,wave_period,wave_height):
    k0 = 1.
    e = 1.
    while e > 1.e-7:
        k1 = k0-(k0*np.tanh(k0*h)-(2.*np.pi/wave_period)**2/9.81)/(np.tanh(k0*h)+k0-k0*(np.tanh(k0*h))**2)
        e = abs((k1-k0)/k0)
        hs = 2.*(np.cosh(2.*k1*h)-1.)/(np.sinh(2.*k1*h)+2*k1*h)
        s = wave_height/hs
        wave_amplitude = 0.5*s
        k0 = k1
    return wave_amplitude

wavemaker(0.3,2.01975,0.02)

0.017511130817405192